## (1) Setup
Go through the setup instruction on our [Github Wiki](https://github.com/pro3d-space/PRo3D/wiki/Interactive-Co%E2%80%90Regsitration-Workflow#setup), which concludes with executing the first cell.

In [ ]:
# imports
from typing import Any
import requests
import json
import open3d as o3d
import numpy as np
import subprocess

from pro3d_api import Pro3DClient
import data_io as io
import o3d_uitls as o3du

In [ ]:
# create api client Pro3D instance
client = Pro3DClient(port=4321)

## (2) Coordinate Alignment
In order to align the coordinate system of SRC to TGT, we need to pick feature points on either surface, that represent the same geologic feature. In our example the angular feature of the cape should work out perfectly. Draw a polyline with **linear projection** on SRC and another on TGT. Both polylines, **must** have the **same number of points**, lie on the **same visual features**, and the points have to be **in the same order**. Minimum number is three, but we recommend to use 5 or 6.

👈 Select annotation drawn on SRC surface

In [ ]:
# Retrieve selected annotation points as SRC - 361
src_points = client.get_selected_annotation_points(verbose=True)

👈  Select annotation drawn on TGT surface

In [ ]:
# Retrieve selected annotation points as TGT - 360
tgt_points = client.get_selected_annotation_points(verbose=True)

In [ ]:
# Compute the rigid transformation matrix using Open3D
alignment_trafo = o3du.compute_rigid_transformation_from_correspondences(tgt_points, src_points)
print(alignment_trafo)

👈 Select TGT surface to be transformed

In [ ]:
# Apply transformation to selected surface
client.apply_transformation_to_selected_surface(alignment_trafo, verbose=True)

In [ ]:
# move to original position
client.apply_transformation_to_selected_surface(np.eye(4), verbose=True)

# (3) Fine Registration

## (3.1) Creating a TGT surface cutout
For fine registration we need to cut out a piece of the target surface to limit the region our source is matched two. For this, we need to draw an annotation on the TGT surface containing the SRC surface. Then we need to select this cutout annotation and the TGT surface we want to cut out from.

👈 select cut out Annotation

👈 select TGT surface

In [ ]:
json_data = client.query_annotation_as_json()
points = np.array(json.loads(json_data))

# Create a point cloud object and assign the points
tgt_pcd = o3du.create_point_cloud_from_array(points)

👈 select SRC surface

In [ ]:
json_data = client.query_annotation_as_json()
points = np.array(json.loads(json_data))

# Create a point cloud object and assign the points
src_pcd = o3du.create_point_cloud_from_array(points)

## (3.2) Employing Iterative Closest Point Optimization

In [ ]:
icp_result = o3du.refine_registration_icp(
    src_pcd, 
    tgt_pcd, 
    alignment_trafo, 
    voxel_size=0.05, 
    max_correspondence_distance=0.075
)

In [ ]:
# Apply final result
client.apply_transformation_to_selected_surface(icp_result.transformation, verbose=True)

In [ ]:
# Apply transformation to selected surface
client.apply_transformation_to_selected_surface(alignment_trafo, verbose=True)

# (4) Validation

In [ ]:
T = icp_result.transformation  # 4x4 matrix

src_pcd.transform(T)

offset = src_pcd.get_center()  # or any reference origin
src_pcd.translate(-offset)
tgt_pcd.translate(-offset)

o3d.io.write_point_cloud("src_transformed.ply", src_pcd)
o3d.io.write_point_cloud("tgt_reference.ply", tgt_pcd)

In [ ]:
o3du.run_m3c2(
    "C:\\Program Files\\CloudCompare\\CloudCompare.exe",
    "src_transformed.ply",
    "tgt_reference.ply",
    "param_file.txt")